In [22]:
import hashlib
from time import time
from uuid import uuid4

class Blockchain(object):
    def __init__(self):
        self.current_transactions = []
        self.chain = []

        # Create the genesis block
        self.new_block(previous_hash=1, proof=100)

    def new_block(self, proof, previous_hash=None):
        """
        Create a new Block in the Blockchain
        :param proof: <int> The proof given by the Proof of Work algorithm
        :param previous_hash: (Optional) <str> Hash of previous Block
        :return: <dict> New Block
        """

        block = {
            'index': len(self.chain) + 1,
            'timestamp': time(),
            'transactions': self.current_transactions,
            'proof': proof,
            'previous_hash': previous_hash or self.hash(self.chain[-1]),
        }

        # Reset the current list of transactions
        self.current_transactions = []

        self.chain.append(block)
        return block

    def new_transaction(self, sender, recipient, amount):
        """
        Creates a new transaction to go into the next mined Block
        :param sender: <str> Address of the Sender
        :param recipient: <str> Address of the Recipient
        :param amount: <int> Amount
        :return: <int> The index of the Block that will hold this transaction
        """
        self.current_transactions.append({
            'sender': sender,
            'recipient': recipient,
            'amount': amount,
        })

        return self.last_block['index'] + 1

    @property
    def last_block(self):
        return self.chain[-1]

    @staticmethod
    def hash(block):
        """
        Creates a SHA-256 hash of a Block
        :param block: <dict> Block
        :return: <str>
        """

        # We must make sure that the Dictionary is Ordered, or we'll have inconsistent hashes
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()
    
    def proof_of_work(self, last_proof):
        """
        Simple Proof of Work Algorithm:
         - Find a number p' such that hash(pp') contains leading 4 zeroes, where p is the previous p'
         - p is the previous proof, and p' is the new proof
        :param last_proof: <int>
        :return: <int>
        """

        proof = 0
        while self.valid_proof(last_proof, proof) is False:
            proof += 1

        return proof

    @staticmethod
    def valid_proof(last_proof, proof):
        """
        Validates the Proof: Does hash(last_proof, proof) contain 4 leading zeroes?
        :param last_proof: <int> Previous Proof
        :param proof: <int> Current Proof
        :return: <bool> True if correct, False if not.
        """

        guess = f'{last_proof}{proof}'.encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        return guess_hash[:4] == "0000"

In [29]:
blockchain = Blockchain()
node_identifier = str(uuid4()).replace('-', '')

def mine():
    # We run the proof of work algorithm to get the next proof...
    last_block = blockchain.last_block
    last_proof = last_block['proof']
    proof = blockchain.proof_of_work(last_proof)

    # We must receive a reward for finding the proof.
    # The sender is "0" to signify that this node has mined a new coin.
    blockchain.new_transaction(
        sender="0",
        recipient=node_identifier,
        amount=1,
    )

    # Forge the new Block by adding it to the chain
    previous_hash = blockchain.hash(last_block)
    block = blockchain.new_block(proof, previous_hash)

    response = {
        'message': "New Block Forged",
        'index': block['index'],
        'transactions': block['transactions'],
        'proof': block['proof'],
        'previous_hash': block['previous_hash'],
    }
    return response



In [37]:
blockchain.chain

[{'index': 1,
  'previous_hash': 1,
  'proof': 100,
  'timestamp': 1516561684.6185923,
  'transactions': []},
 {'index': 2,
  'previous_hash': '25f487820edd334f3ba52a8bb9e5b1b4f9077bea7e183d5774ea1ee7da683bdf',
  'proof': 35293,
  'timestamp': 1516561687.9184039,
  'transactions': [{'amount': 1,
    'recipient': '6228c1bb6251489f94c8cc55a93d5812',
    'sender': '0'}]},
 {'index': 3,
  'previous_hash': '185ce803d0775d1a22de7780025d815f65412e4fb001f47452467c0d80c8e2a2',
  'proof': 35089,
  'timestamp': 1516561720.4652727,
  'transactions': [{'amount': '10', 'recipient': 'someone', 'sender': 'moi'},
   {'amount': 1,
    'recipient': '6228c1bb6251489f94c8cc55a93d5812',
    'sender': '0'}]}]

In [34]:
mine()

{'index': 3,
 'message': 'New Block Forged',
 'previous_hash': '185ce803d0775d1a22de7780025d815f65412e4fb001f47452467c0d80c8e2a2',
 'proof': 35089,
 'transactions': [{'amount': '10', 'recipient': 'someone', 'sender': 'moi'},
  {'amount': 1,
   'recipient': '6228c1bb6251489f94c8cc55a93d5812',
   'sender': '0'}]}

In [33]:
full_chain()

{'chain': [{'index': 1,
   'previous_hash': 1,
   'proof': 100,
   'timestamp': 1516561684.6185923,
   'transactions': []},
  {'index': 2,
   'previous_hash': '25f487820edd334f3ba52a8bb9e5b1b4f9077bea7e183d5774ea1ee7da683bdf',
   'proof': 35293,
   'timestamp': 1516561687.9184039,
   'transactions': [{'amount': 1,
     'recipient': '6228c1bb6251489f94c8cc55a93d5812',
     'sender': '0'}]}],
 'length': 2}

In [32]:
new_transaction('moi','someone', '10')

{'message': 'Transaction will be added to Block 3'}